In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
sys.path.insert(0,'../slim/')
import os
import json
import math
import time
import numpy as np
import tensorflow as tf

In [4]:
from nets import nets_factory
from preprocessing import preprocessing_factory
from PIL import Image
import cv2
slim = tf.contrib.slim

In [5]:
import pandas as pd

In [89]:
tf.logging.set_verbosity(tf.logging.INFO)

In [47]:
filenames = pd.read_csv('valid_image_paths.csv',header=None).iloc[:,0].tolist()

In [ ]:
'MURA-v1.1/valid/XR_ELBOW/patient'

In [29]:
for filename in filenames:
    image=cv2.imread(filename)
    res=cv2.resize(image,(299,299),interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(filename, res)

In [30]:
result = {}
with tf.Graph().as_default():
        tf_global_step = slim.get_or_create_global_step()
        network_fn = nets_factory.get_network_fn(
        'inception_resnet_v2',
        num_classes=(2),
        is_training=False)
        preprocessing_name = 'inception_resnet_v2'
        image_preprocessing_fn = preprocessing_factory.get_preprocessing(
            preprocessing_name,
            is_training=False)
        test_image_size = network_fn.default_image_size
        checkpoint_path = tf.train.latest_checkpoint('/home/paperspace/stanford/train_logs/')
        batch_size = 1
        tensor_input = tf.placeholder(tf.float32, [None, test_image_size, test_image_size, 3])
        logits, _ = network_fn(tensor_input)
        logits = tf.nn.top_k(logits, 2)
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()
            saver.restore(sess, checkpoint_path)
            for filename in filenames:
                image = tf.read_file(filename)
                image = tf.image.decode_jpeg(image, channels=0)
                processed_image = image_preprocessing_fn(image, test_image_size, test_image_size)
                processed_image = sess.run(processed_image)
                images = np.array([processed_image])
                predictions = sess.run(logits, feed_dict = {tensor_input : images})
                pre = predictions.values.argmax()
                result[filename] = pre

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/paperspace/stanford/train_logs/model.ckpt-171573


In [31]:
result

{'../data_all/negative/patient00011_study1_negative_image2.png': 0,
 '../data_all/positive/patient00003_study1_positive_image1.png': 0}

In [33]:
result

{'../data_all/negative/patient00011_study1_negative_image2.png': 0,
 '../data_all/positive/patient00003_study1_positive_image1.png': 0}

In [35]:
result.keys()

dict_keys(['../data_all/negative/patient00011_study1_negative_image2.png', '../data_all/positive/patient00003_study1_positive_image1.png'])

In [44]:
with open('output.csv','w') as file:
    for i in list(result.keys()):
        file.write(i+',')
        file.write(str(result[i]))
        file.write('\n')